# Election Write-In Vote Analysis for the 2018 Alabama Special Election

SUMMARY

I had heard that the Alabama special election had an advanced number of write-in votes from voters who did not want to vote for either candidates.  I then decided to collect the voter data published by the Alambama Secretary of State and to perform an analysis of the number of write-in votes for each county.  In particular I wanted to know the tally of and percentage of write-in votes for each county.


In [12]:
library(ggplot2)
library(maps)
library(plyr)
library(dplyr)

dbg <- FALSE


I want to display the results of my analysis for each county in the state, which meant that I needed the approximate geospatial coordinates for each county.  I searched to see if they were available but I did not find them right away. I ended up manually determining the center of each county and then creating a text file of the inputs.  This file is included in this notebook, which I'll read now for the analysis:


In [13]:
countyGPSCoords<-read.csv('./AlabamaCountyGPSCenters.txt',sep=',')
if (dbg == TRUE) {
    countyGPSCoords
} else {
    head(countyGPSCoords)
}

County,Lat,Long
Autauga,32.53734,-86.64368
Baldwin,30.57564,-87.68228
Barbour,31.86184,-85.41044
Bibb,32.96198,-87.14914
Blount,33.99339,-86.54842
Bullock,32.11306,-85.72538



Load the exported voting data published by the Secretary of State.  As you can see from the start of the data there were recorded votes for the candidates Doug Jones and Roy S. Moore, as well as a tallied count of write-in votes.  there were also some other elections carried out at various counties, such as school tax millages, but these will be filtered out later in the analysis.


In [14]:
sosEnrExport <- read.csv('./sosEnrExport.csv', sep = ',')
if (dbg == TRUE) {
    sosEnrExport
} else {
    head(sosEnrExport)
}

Election.Code,Election.Title,County.Code,County.Name,Contest.Code,Contest.Title,Candidate.Number,Candidate.Name,Votes,Party.Code
1000915,U.S. SENATE SPECIAL GENERAL ELECTION 2017,1,Jefferson,1000900,UNITED STATES SENATOR,1,Doug Jones,149522,DEM
1000915,U.S. SENATE SPECIAL GENERAL ELECTION 2017,1,Jefferson,1000900,UNITED STATES SENATOR,2,Roy S. Moore,66309,REP
1000915,U.S. SENATE SPECIAL GENERAL ELECTION 2017,1,Jefferson,1000900,UNITED STATES SENATOR,3,Write-In,3710,NON
1000915,U.S. SENATE SPECIAL GENERAL ELECTION 2017,2,Mobile,1000900,UNITED STATES SENATOR,1,Doug Jones,62253,DEM
1000915,U.S. SENATE SPECIAL GENERAL ELECTION 2017,2,Mobile,1000900,UNITED STATES SENATOR,2,Roy S. Moore,46725,REP
1000915,U.S. SENATE SPECIAL GENERAL ELECTION 2017,2,Mobile,1000900,UNITED STATES SENATOR,3,Write-In,1539,NON



First I'll select only the write-in data from the election results of the U. S. Senate Special General Election, which is identified as contest title 1000900:


In [15]:
writeInVotes <- sosEnrExport %>%
  filter(Contest.Code == 1000900 & Candidate.Name == 'Write-In') %>%
  select(County.Name, Votes) %>%
  arrange(County.Name)
if (dbg == TRUE) {
  writeInVotes
} else {
    head(writeInVotes)
}

County.Name,Votes
Autauga,253
Baldwin,1699
Barbour,41
Bibb,66
Blount,180
Bullock,7



Let's determine the number of voters submitted for each county, which will include not only the votes for the two main candidates but the write-in votes as well:


In [16]:
totalVoters <- sosEnrExport %>%
  filter(Contest.Code == 1000900) %>%
  select(County.Name, Votes) %>%
  group_by(County.Name, add = FALSE) %>%
  summarize(TotalCountyVoters = sum(Votes)) %>%
  arrange(County.Name)
if (dbg == TRUE) {
    totalVoters
} else {
    head(totalVoters)
}

County.Name,TotalCountyVoters
Autauga,14611
Baldwin,62275
Barbour,6420
Bibb,5232
Blount,14206
Bullock,3375



Now I'll join the write-in vote data frame with the county coordinates for plotting the results.  To that data frame I'll also join the total voter results:


In [17]:
writeInVotesWithCoords<-left_join(writeInVotes,countyGPSCoords,by=c("County.Name"="County"))
writeInVotesWithCoordsAndTotalVoters<-left_join(writeInVotesWithCoords,totalVoters,by=c("County.Name"="County.Name"))

writeInVotesWithCoordsPct<-writeInVotesWithCoordsAndTotalVoters %>%
  mutate(PctWriteIns=(Votes/TotalCountyVoters*100))
if (dbg == TRUE) {
    writeInVotesWithCoordsPct
} else {
    head(writeInVotesWithCoordsPct)
}

County.Name,Votes,Lat,Long,TotalCountyVoters,PctWriteIns
Autauga,253,32.53734,-86.64368,14611,1.7315721
Baldwin,1699,30.57564,-87.68228,62275,2.7282216
Barbour,41,31.86184,-85.41044,6420,0.6386293
Bibb,66,32.96198,-87.14914,5232,1.2614679
Blount,180,33.99339,-86.54842,14206,1.2670703
Bullock,7,32.11306,-85.72538,3375,0.2074074



Now I can plot the percentage of write-in votes for each county:


In [18]:
ggplot(data=writeInVotesWithCoordsPct)+
  borders(database='county',region='alabama',colour='grey60',fill='grey90')+
  geom_point(aes(x=Long,y=Lat,size=PctWriteIns),color='blue',fill='blue')+
  scale_size_area(max_size=3)+
  ggtitle('Percentage of Write-In Votes by County in Alabama')+
  xlab('')+
  ylab('')+
  theme(
    panel.background=element_blank(),
    axis.title.x=element_blank(),
    axis.text.x=element_blank(),
    axis.ticks.x=element_blank(),
    axis.title.y=element_blank(),
    axis.text.y=element_blank(),
    axis.ticks.y=element_blank()) 


You can see the trend that the higher number of write-in votes were clustered around the north and north-east portions of the state.  There were fewer write-in votes in the south-west corner of the state.  In fact, most of the south of the state had fewer write-in votes with the exception of the city of Mobile.

Now let's plot the count of write-in votes for each county:


In [19]:
ggplot(data=writeInVotesWithCoords)+
  borders(database='county',region='alabama',colour='grey60',fill='grey90')+
  geom_point(aes(x=Long,y=Lat,size=Votes),color='darkgreen',fill='darkgreen')+
  scale_size_area(max_size=8)+
  ggtitle('Count of Write-In Votes by County in Alabama')+
  xlab('')+
  ylab('')+
  theme(
    panel.background=element_blank(),
    axis.title.x=element_blank(),
    axis.text.x=element_blank(),
    axis.ticks.x=element_blank(),
    axis.title.y=element_blank(),
    axis.text.y=element_blank(),
    axis.ticks.y=element_blank()) 


Here we can see that the tally of write-in votes is clearly highest around the higher populated cities in the state, e.g. Huntsville, Birmingham, and Mobile.
